In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras import regularizers
import keras
import numpy as np
from keras.utils import plot_model
classes = ["arisa", "tisato"]
num_classes = len(classes)



# メインの関数を定義する
def main():
    X_train, X_test, y_train, y_test = np.load("./member2.npy")
    X_train = X_train.astype("float") / 256
    X_test = X_test.astype("float") / 256
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)

    model = model_train(X_train, y_train, X_test, y_test)

def model_train(X, y,X_valid, y_valid):
    input_shape = (126, 167, 3)  # モデルの入力サイズ
    #num_classes = 26  # クラス数

    # モデルを作成する。
    # model = ResnetBuilder.build_resnet_18(input_shape, num_classes)
    model = Sequential()
    model.add(Conv2D(64,(3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(128,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    

    opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    fit = model.fit(X, y,validation_data=(X_valid, y_valid), batch_size=32, epochs=30)
    
    # モデルの保存
    model.save('./member_cnn2.h5')

    return model

def model_eval(model, X, y):
    scores = model.evaluate(X, y, verbose=1)
    print('Test Loss: ', scores[0])
    print('Test Accuracy: ', scores[1])

if __name__ == "__main__":
    main()

Train on 147 samples, validate on 1 samples
Epoch 1/30
147/147 [==============================] - 2s 15ms/step - loss: 1.1008 - accuracy: 0.5170 - val_loss: 0.2668 - val_accuracy: 1.0000
Epoch 2/30
147/147 [==============================] - 1s 4ms/step - loss: 1.3193 - accuracy: 0.5238 - val_loss: 1.0405 - val_accuracy: 0.0000e+00
Epoch 3/30
147/147 [==============================] - 0s 3ms/step - loss: 0.9305 - accuracy: 0.4422 - val_loss: 0.7533 - val_accuracy: 0.0000e+00
Epoch 4/30
147/147 [==============================] - 0s 3ms/step - loss: 0.7054 - accuracy: 0.4830 - val_loss: 0.6484 - val_accuracy: 1.0000
Epoch 5/30
147/147 [==============================] - 0s 3ms/step - loss: 0.6922 - accuracy: 0.5510 - val_loss: 0.6630 - val_accuracy: 1.0000
Epoch 6/30
147/147 [==============================] - 0s 3ms/step - loss: 0.6786 - accuracy: 0.5510 - val_loss: 0.6771 - val_accuracy: 1.0000
Epoch 7/30
147/147 [==============================] - 0s 3ms/step - loss: 0.6810 - accuracy: 0.

In [6]:
from PIL import Image
import os, glob
import numpy as np
from keras.models import load_model

input_dir = 'test'

X = []


files = glob.glob(input_dir + "/*.jpg")
for file in files:
    image = Image.open(file)
    image = image.convert("RGB")
    image = image.resize((167, 126))
    data = np.asarray(image)
    X.append(data)

X = np.array(X)
model = load_model('./member_cnn2.h5')

predict = model.predict(X)
print(predict)

[[0. 1.]]
